In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [3]:
import netCDF4 as nc
import numpy as np
import arrow
import glob

## Purpose of notebook

Hi Evie, 

I'm writing you some code to extract data of interest from the 4 years of our hindcast. You would like timeseries of pH and $\Omega_A$ for the whole watercolumn for the following predefined places:

    list_places = ['Deep Bay', 'Southern Baynes', 'Northern Baynes', 
    'Fanny Bay', 'Maple Bay', 'Salt Spring', 'Nanoose Bay', 'Lasqueti Island', 
    'Main SoG','Cortes/Marina', 'Lund/Desolation Sound', 'Mouth of Okeover']

As a review, our model outputs carbonate chemistry in the form of DIC and TA, in very large .nc files [(here is more info on .nc files)](https://www.unidata.ucar.edu/software/netcdf/). One day's worth is around a gigabyte. Biologically relevant variables such as pH and $\Omega_A$ are then backcalculated from these (for this, you also need T and S). In this notebook (A), I'm going to show you code for extracting those data (DIC, TA, S, T) from our model output for all the places you want for a given year and saving it a different , much smaller, file - one file per day. In notebook B, I'll consolidate these daily small files into one bigger file. In notebook C, I'll then calculate pH and Omega and plot them. I've done my best to comment my code using #TJSJ (from my initials TJŠJ) to show what I'm doing. 

This code shows how to extract the relevant data for all stations for a given day and save it in a .nc file

#### TJSJ - first I make a dictionary of your places with lat, lon, and model coordinates. 
    
    Original dictionary by E.Olson (I think)

In [4]:


PLACES = {
'Deep Bay': {

# deg E, deg N

'lon lat': (-124.7392, 49.4606),

# indices of nearest NEMO model grid point

# j is the latitude (y) direction, i is the longitude (x) direction

'NEMO grid ji': (599, 126),

},

'Southern Baynes': {

'lon lat': (-124.7457, 49.4760),

'NEMO grid ji': (602, 127),

},

'Northern Baynes': {

'lon lat': (-124.8924, 49.6492),

'NEMO grid ji': (646, 127),

},

'Fanny Bay': {

'lon lat': (-124.8227, 49.5086),

'NEMO grid ji': (614, 120),

},

'Maple Bay': {

'lon lat': (-123.5947, 48.8140),

'NEMO grid ji': (392, 213),

},

'Salt Spring': {

'lon lat': (-123.5513, 48.7993),

'NEMO grid ji': (386, 218),

},

'Nanoose Bay': {

'lon lat': (-124.1359, 49.2609),

'NEMO grid ji': (517, 190),

},

'Lasqueti Island': {

# deg E, deg N

'lon lat': (-124.3384, 49.5442),

'NEMO grid ji': (586, 195),

},

'Main SoG': {

'lon lat': (-123.5832, 49.1177),

'NEMO grid ji': (450, 253),

},

'Cortes/Marina': {

'lon lat': (-125.0194, 50.0418),

'NEMO grid ji': (732, 157),

},

'Lund/Desolation Sound': {

'lon lat': (-124.7666, 49.9804),

'NEMO grid ji': (702, 187),

},

'Mouth of Okeover': {

'lon lat': (-124.8174, 50.0805),

'NEMO grid ji': (726, 192),

},

}



#### TJSJ I make a list of place names, for calling the dictionary from
    
    then I test that I can access the data from the dictionary 



In [5]:
list_places = ['Deep Bay', 'Southern Baynes', 'Northern Baynes', \
               'Fanny Bay', 'Maple Bay', 'Salt Spring', 'Nanoose Bay',\
               'Lasqueti Island', 'Main SoG', 'Cortes/Marina', \
               'Lund/Desolation Sound', 'Mouth of Okeover']

for i in range(0,len(list_places)):
    t_place = (list_places[i])
    print('Place is '+t_place +\
          '. The j,i coords are '+str(PLACES[t_place]['NEMO grid ji']))

Place is Deep Bay. The j,i coords are (599, 126)
Place is Southern Baynes. The j,i coords are (602, 127)
Place is Northern Baynes. The j,i coords are (646, 127)
Place is Fanny Bay. The j,i coords are (614, 120)
Place is Maple Bay. The j,i coords are (392, 213)
Place is Salt Spring. The j,i coords are (386, 218)
Place is Nanoose Bay. The j,i coords are (517, 190)
Place is Lasqueti Island. The j,i coords are (586, 195)
Place is Main SoG. The j,i coords are (450, 253)
Place is Cortes/Marina. The j,i coords are (732, 157)
Place is Lund/Desolation Sound. The j,i coords are (702, 187)
Place is Mouth of Okeover. The j,i coords are (726, 192)


#### TJSJ Write a function that returns a list of paths to nc files for given days in a year. 

    To do that I basically define my start date and end date (see below) 
    and put all the dates between them in different formats using the arrow
    function, then string together a string name based on what kind of data
    I want (carbon or Temp/Sal). By convention that Susan made up, we save
    the data in two filename -suffixes
    
        grid_T, (short for tracers on the T grid) which contains T and S 
        carp_T, (short for carbon plus) which contains DIC, TA, and other things

    in the verbose option, the function spits out the dateformats 
    and what filename it's found for each date
    below I test it for two days to make sure it's doing what I think it is 
    and print the resulting array of ncs

In [6]:
start = '2016-01-01'
end = '2016-01-02'
verbose = True
ftype = 'carp'

def make_nclist(start,end,ftype,verbose):

    print('Variable type is '+ftype)
    print('')
    nc_array = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    
    
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        if verbose == True:
            print('DATE IS: ' +str(tdate))
        ddmmmyy = tdate.format('DDMMMYY').lower()
        if verbose == True:
            print('Arrow ddmmmyy format is: '+ str(ddmmmyy))
        ymd = tdate.format('YYYYMMDD')
        if verbose == True:
            print('Arrow ymd format is: '+ str(ymd))
            
        t_nc = '/results/SalishSea/hindcast.201905/' \
        + str(ddmmmyy)+ '/SalishSea_*' + str(ymd) + '_' + str(ymd) + '_' + ftype + '_T.nc'
        t_ncname = glob.glob(t_nc)
        if verbose == True:
            print('file is: ' + t_ncname[0])
            print('')
        nc_array.append(t_ncname[0])
        
    return nc_array

nc_array = make_nclist(start,end,ftype,verbose)

print(nc_array)

Variable type is carp

DATE IS: 2016-01-01T00:00:00+00:00
Arrow ddmmmyy format is: 01jan16
Arrow ymd format is: 20160101
file is: /results/SalishSea/hindcast.201905/01jan16/SalishSea_1h_20160101_20160101_carp_T.nc

DATE IS: 2016-01-02T00:00:00+00:00
Arrow ddmmmyy format is: 02jan16
Arrow ymd format is: 20160102
file is: /results/SalishSea/hindcast.201905/02jan16/SalishSea_1h_20160102_20160102_carp_T.nc

['/results/SalishSea/hindcast.201905/01jan16/SalishSea_1h_20160101_20160101_carp_T.nc', '/results/SalishSea/hindcast.201905/02jan16/SalishSea_1h_20160102_20160102_carp_T.nc']


#### TJSJ here I'm extracting the filenames for the year 2013 

    for both the carbon (carp) and T,S (grid) filenames.
    verbose set to False to avoid it vomiting output at me. 


In [7]:
start = '2013-01-01'
end = '2013-12-31'
verbose = False
ftype = 'carp'

ncs_carp = make_nclist(start,end,ftype,verbose)

ftype = 'grid'
ncs_grid = make_nclist(start,end,ftype,verbose)

Variable type is carp

Variable type is grid



#### Now I'm making a dictionary to store data for top 3m for all stations for any given day

    It's another dictionary, PLACES_withdat
    I initialized it with numpy arrays of zeroes for the 4 variables I need .
    each array is of size 3, for 3 depths

In [8]:
PLACES_withdat = {
'Deep Bay': {
'NEMO grid ji': (599, 126),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),

},
    
'Southern Baynes': {
'NEMO grid ji': (602, 127),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),

},

'Northern Baynes': {
'NEMO grid ji': (646, 127),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

'Fanny Bay': {
'NEMO grid ji': (614, 120),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

'Maple Bay': {
'NEMO grid ji': (392, 213),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

'Salt Spring': {
'NEMO grid ji': (386, 218),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

'Nanoose Bay': {
'NEMO grid ji': (517, 190),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

'Lasqueti Island': {
'NEMO grid ji': (586, 195),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

'Main SoG': {
'NEMO grid ji': (450, 253),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

'Cortes/Marina': {
'NEMO grid ji': (732, 157),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

'Lund/Desolation Sound': {
'NEMO grid ji': (702, 187),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

'Mouth of Okeover': {
'NEMO grid ji': (726, 192),
    'T_timeseries':np.zeros([3]),
    'S_timeseries':np.zeros([3]),
    'DIC_timeseries':np.zeros([3]),
    'TA_timeseries':np.zeros([3]),
},

}


#### TJSJ Now let's open the carp and grid dataset for the first day of the year

    first day of year is jan 1st 2013, 0th index in our array
    let's just print out our filename and what's in it.


In [9]:

day = 0
#TJSJ carbon data
print('filename is: ')
print(ncs_carp[0])
print()
t_carp = nc.Dataset(ncs_carp[0])
#print what's in the dataset
print(t_carp)
#TJSJ grid data
print('filename is: ')
print(ncs_grid[0])
print()
t_grid = nc.Dataset(ncs_grid[0])
#print what's in the dataset
print(t_grid)



filename is: 
/results/SalishSea/hindcast.201905/01jan13/SalishSea_1h_20130101_20130101_carp_T.nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    name: SalishSea_1h_20130101_20130105
    description: auxilary variables
    title: auxilary variables
    Conventions: CF-1.6
    timeStamp: 2019-Sep-14 19:05:50 GMT
    uuid: a850fe2d-5636-424e-b1a7-fb49f3240f05
    dimensions(sizes): axis_nbounds(2), x(398), y(898), nvertex(4), deptht(40), time_counter(24)
    variables(dimensions): float32 nav_lat(y,x), float32 nav_lon(y,x), float32 bounds_lon(y,x,nvertex), float32 bounds_lat(y,x,nvertex), float32 area(y,x), float32 deptht(deptht), float32 deptht_bounds(deptht,axis_nbounds), float32 PAR(time_counter,deptht,y,x), float64 time_centered(time_counter), float64 time_centered_bounds(time_counter,axis_nbounds), float64 time_counter(time_counter), float64 time_counter_bounds(time_counter,axis_nbounds), float32 sigma_theta(time_counter,deptht,y,x), float3

#### TJSJ now let's take out the data, check its size, and then average it to daily from hourly

   variable size is (24, 40, 898, 398): 24 hours, 40 depths, 898, 398
   daily averaged variable size is (40, 898, 398)

In [10]:

SAL = t_grid['vosaline']
TEMP = t_grid['votemper']
DIC = t_carp['dissolved_inorganic_carbon']
TA = t_carp['total_alkalinity']

print('shape of SAL variable')
print(np.shape(SAL))

SAL_daily = np.mean(SAL,axis=0)
TEMP_daily = np.mean(TEMP,axis=0)
DIC_daily = np.mean(DIC,axis=0)
TA_daily = np.mean(TA,axis=0)
print('shape of SAL_daily')
print(np.shape(SAL_daily))




shape of SAL variable
(24, 40, 898, 398)
shape of SAL_daily
(40, 898, 398)


#### TJSJ loop through list of stations, storing T,S,DIC,TA in the dictionary we made

    as a check, I'm printing out the DIC values for the stations we're looking for

In [11]:
for i in range(0,len(list_places)):
    t_place = list_places[i]
    t_j_index = PLACES[t_place]['NEMO grid ji'][0]
    t_i_index = PLACES[t_place]['NEMO grid ji'][1]
    print('Station is: ' + t_place + '. j is: '+str(t_j_index)+', i is: '+str(t_i_index))
    
    t_DIC = DIC_daily[0:3,t_j_index,t_i_index]
    t_TA = TA_daily[0:3,t_j_index,t_i_index]
    t_SAL = SAL_daily[0:3,t_j_index,t_i_index]
    t_TEMP = TEMP_daily[0:3,t_j_index,t_i_index]
    print('DIC values (top 3m) are: '+str(t_DIC))
    print('TA values (top 3m) are: '+str(t_TA))
    print()
    
    PLACES_withdat[t_place]['T_timeseries'] = t_TEMP
    PLACES_withdat[t_place]['S_timeseries'] = t_SAL
    PLACES_withdat[t_place]['DIC_timeseries'] = t_DIC
    PLACES_withdat[t_place]['TA_timeseries'] = t_TA

Station is: Deep Bay. j is: 599, i is: 126
DIC values (top 3m) are: [1727.4579 1848.1748 1950.4027]
TA values (top 3m) are: [1775.049  1902.7909 2010.8522]

Station is: Southern Baynes. j is: 602, i is: 127
DIC values (top 3m) are: [1794.9349 1929.7383 1968.4567]
TA values (top 3m) are: [1847.9092 1989.7203 2029.9442]

Station is: Northern Baynes. j is: 646, i is: 127
DIC values (top 3m) are: [1745.107  1972.7373 1995.0317]
TA values (top 3m) are: [1789.4132 2029.889  2053.1213]

Station is: Fanny Bay. j is: 614, i is: 120
DIC values (top 3m) are: [1676.3019 1860.8667 1967.1818]
TA values (top 3m) are: [1720.3026 1919.1771 2031.0568]

Station is: Maple Bay. j is: 392, i is: 213
DIC values (top 3m) are: [1897.8878 1920.7955 1934.87  ]
TA values (top 3m) are: [1966.7697 1990.2909 2004.5367]

Station is: Salt Spring. j is: 386, i is: 218
DIC values (top 3m) are: [1933.167  1964.1224 1973.9298]
TA values (top 3m) are: [2002.1011 2032.6025 2041.6934]

Station is: Nanoose Bay. j is: 517, i i

#### TJSJ now I'll name an nc filename and store the data in it

    is there a better/more elegant way to 
    write the code that writes the data to the .nc file?
    almost definitely
    don't know what it is though 
    and can't be bothered googling stackoverflow for a long time
    to try to figure out

In [12]:
#TJSJ
str_name = 'EVIE_PLACES_TEST3_'
ymd = '01jan13'
t_str = str_name + ymd + '.nc'
print('our new .nc file will be named:')
print(t_str)




our new .nc file will be named:
EVIE_PLACES_TEST3_01jan13.nc


In [13]:


f = nc.Dataset(t_str, 'w')
g = f.createGroup('Deep Bay')
g.createDimension('depths', 3)
g.createDimension('days',365)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Deep Bay']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Deep Bay']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Deep Bay']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Deep Bay']['TA_timeseries'][:]

g = f.createGroup('Southern Baynes')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Southern Baynes']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Southern Baynes']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Southern Baynes']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Southern Baynes']['TA_timeseries'][:]

g = f.createGroup('Northern Baynes')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Northern Baynes']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Northern Baynes']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Northern Baynes']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Northern Baynes']['TA_timeseries'][:]

g = f.createGroup('Fanny Bay')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Fanny Bay']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Fanny Bay']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Fanny Bay']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Fanny Bay']['TA_timeseries'][:]

g = f.createGroup('Maple Bay')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Maple Bay']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Maple Bay']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Maple Bay']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Maple Bay']['TA_timeseries'][:]

g = f.createGroup('Salt Spring')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Salt Spring']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Salt Spring']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Salt Spring']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Salt Spring']['TA_timeseries'][:]

g = f.createGroup('Nanoose Bay')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Nanoose Bay']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Nanoose Bay']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Nanoose Bay']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Nanoose Bay']['TA_timeseries'][:]

g = f.createGroup('Lasqueti Island')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Lasqueti Island']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Lasqueti Island']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Lasqueti Island']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Lasqueti Island']['TA_timeseries'][:]

g = f.createGroup('Main SoG')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Main SoG']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Main SoG']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Main SoG']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Main SoG']['TA_timeseries'][:]

g = f.createGroup('Cortes/Marina')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Cortes/Marina']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Cortes/Marina']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Cortes/Marina']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Cortes/Marina']['TA_timeseries'][:]

g = f.createGroup('Lund/Desolation Sound')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Lund/Desolation Sound']['TA_timeseries'][:]

g = f.createGroup('Mouth of Okeover')
g.createDimension('depths', 3)
ts = g.createVariable('T_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Mouth of Okeover']['T_timeseries'][:]
ts = g.createVariable('S_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Mouth of Okeover']['S_timeseries'][:]
ts = g.createVariable('DIC_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Mouth of Okeover']['DIC_timeseries'][:]
ts = g.createVariable('TA_timeseries','f4',('depths'))
ts[:] = PLACES_withdat['Mouth of Okeover']['TA_timeseries'][:]


f.close()

#### TJSJ let's open the .nc file we just wrote and make sure it works

In [14]:
w = nc.Dataset(t_str)
print(w['Deep Bay'])
print(w['Deep Bay']['DIC_timeseries'][:])
print(w['Deep Bay']['TA_timeseries'][:])

<class 'netCDF4._netCDF4.Group'>
group /Deep Bay:
    dimensions(sizes): depths(3), days(365)
    variables(dimensions): float32 T_timeseries(depths), float32 S_timeseries(depths), float32 DIC_timeseries(depths), float32 TA_timeseries(depths)
    groups: 

[1727.4579 1848.1748 1950.4027]
[1775.049  1902.7909 2010.8522]
